## Asynchronous LLM calls

*[Coding along with the Udemy Course [Advanced Retrieval Augmented Generation ](https://www.udemy.com/course/advanced-retrieval-augmented-generation/) by Rémi Connesson]*

Calling the OpenAI API concurrently instead of sequentially.

In [1]:
import pandas as pd
from openai import AsyncOpenAI

In [2]:
api_key = pd.read_csv("~/tmp/chat_gpt/agentic-design-1.txt", sep=" ", header=None)[0][0]
print("Don't be a fool and sent your api key to github")

Don't be a fool and sent your api key to github


In [3]:
client = AsyncOpenAI(api_key=api_key)

In [4]:
def _msg(role, content):
    return {'role': role, 'content': content}

def system(content):
    return _msg('system', content)

def user(content):
    return _msg('user', content)

def assistant(content):
    return _msg('assistant', content)

In [5]:
chat_history = [
    system("You are a helpful assistant."),
    user("Hey, what is Python?"),
    assistant("Python is a good programming language"),
    user("Why so....???"),
]

In [6]:
from pprint import pprint

In [7]:
pprint(chat_history)

[{'content': 'You are a helpful assistant.', 'role': 'system'},
 {'content': 'Hey, what is Python?', 'role': 'user'},
 {'content': 'Python is a good programming language', 'role': 'assistant'},
 {'content': 'Why so....???', 'role': 'user'}]


In [8]:
completion = await client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = chat_history,
    max_tokens = 50 # limit the output to safe costs
) # returns a coroutine that's an awaitable so we need to put an await in front
# <coroutine object AsyncCompletions.create at 0x1039706c0>

In [9]:
completion # returns a coroutine that's an awaitable so we need to put an await in front

ChatCompletion(id='chatcmpl-ANIDyrA0YgJWh7sjiRhHbjNk921sY', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a good programming language for several reasons:\n\n1. **Readability and Simplicity**: Python has a clean and easy-to-read syntax that resembles natural language, which makes it accessible for beginners. It emphasizes readability, which makes it', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730114962, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f59a81427f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

In [10]:
# making completions work concurrently with Python's asyncio library
import asyncio

In [11]:
# making completions work concurrently
completion_task_1 = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = chat_history,
    max_tokens = 50 # limit the output to safe costs
)

completion_task_2 = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = chat_history,
    max_tokens = 50 # limit the output to safe costs
)

In [12]:
completion_task_1, completion_task_2 # two coroutines

(<coroutine object AsyncCompletions.create at 0x11e336600>,
 <coroutine object AsyncCompletions.create at 0x11e337c80>)

In [13]:
# execution calls concurrently with asyncio's gather function
results = await asyncio.gather(completion_task_1, completion_task_2)

In [14]:
results

[ChatCompletion(id='chatcmpl-ANIHuFwVWnC9RQ3c00NnHqDDASHL9', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="Python is considered a good programming language for several reasons:\n\n1. **Readability and Simplicity**: Python's syntax is designed to be clear and easy to read, which makes it accessible to beginners. This readability allows programmers to express concepts in", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730115206, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f59a81427f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))),
 ChatCompletion(id='chatcmpl-ANIHuONSdajtlpXEUQ3sM4n6pNOH1', choices=[Choice(finish_reason='length', i

In [15]:
# now let's just create a list of tasks
completion_tasks = []

for i in range(50):
    completion_task = client.chat.completions.create(
        model = "gpt-4o-mini",
        messages = chat_history,
        max_tokens = 50 # limit the output to safe costs
    )

    completion_tasks.append(completion_task)

In [16]:
len(completion_tasks) # list of coroutines

50

In [17]:
results_list = await asyncio.gather(*completion_tasks) # unpack them all with *

In [18]:
results_list

[ChatCompletion(id='chatcmpl-ANIKZh8uXBrk2HurJJAEwaCfAjWDO', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a good programming language for several reasons:\n\n1. **Readability and Simplicity**: Python has a clear and straightforward syntax that emphasizes readability. This makes it easier for both beginners and experienced programmers to understand and write code effectively.\n\n', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730115371, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f59a81427f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))),
 ChatCompletion(id='chatcmpl-ANIKZPzPyker3Dlv0XO2rtU6RFChC', choices=[

In [19]:
results_list[-1] # having a look at the last one

ChatCompletion(id='chatcmpl-ANIKaUkzQ3ToeboJCOGcCotAvhjCS', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Python is considered a good programming language for several reasons:\n\n1. **Easy to Learn and Use**: Python has a simple and clear syntax that makes it accessible for beginners. The readability of Python code allows new programmers to quickly grasp programming concepts.\n\n2', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1730115372, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_f59a81427f', usage=CompletionUsage(completion_tokens=50, prompt_tokens=41, total_tokens=91, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))